In [52]:
import obspy
import obspy.signal.filter
from obspy.io.sac.sactrace import SACTrace
from obspy.io.sac.util import get_sac_reftime
from obspy.clients.iris import Client
from obspy.taup import TauPyModel
import os
import glob
import numpy as np
import shutil
import pickle
from obspy import read
import random
import pandas as pd
import sys
import re

In [53]:
client = Client()
hk_model = f"{ddir}/hk.npz"
model = TauPyModel(model= hk_model)

In [18]:
os.chdir("/mnt")
ddir = f"/mnt/c/Users/youzh/Documents/My documents(not synced)/sP_Workflow"
data_sac = f"{ddir}/Download/SAC"
wdir = f"{ddir}/Download/Verified"
jdir = f"{ddir}/judge"
wsl_dir = f"/home/youzhi/python-projects/sp-project"
os.getcwd()

'/mnt'

# Convert to envelope, normalize and cut waveform

In [20]:
df_verified = pd.read_csv(f"{wdir}/verified.txt", names=["evt_id","sta", "Tp", 
                                                   "p_len","TsPmP", "sPmP_len",
                                                   "TsPn", "sPn_len", "quality", "evt_num"], header = None)

df_verified['evt_num'] = df_verified['evt_num'].astype(str)
df_verified

,evt_id,sta,Tp,p_len,TsPmP,sPmP_len,TsPn,sPn_len,quality,evt_num
0,39909423,CI.HOL.,15.558,0.3,18.173,0.3,nan,0.4,a,20220112002458770
1,39909511,AZ.LVA2.,3.723,0.2,11.662,0.3,nan,0.4,b,20220112021208390
2,39909511,AZ.SMER.,12.748,0.2,16.068,0.3,nan,0.4,b,20220112021208390
3,39909511,BC.RMX.,16.035,0.3,19.083,0.3,18.483,0.3,a,20220112021208390
4,39909663,CI.EML.,18.840,0.4,23.763,0.4,22.793,0.4,a,20220112090603170
...,...,...,...,...,...,...,...,...,...,...
100,40183624,CI.SLH.,3.360,0.4,13.400,0.4,nan,0.3,b,20220211232548160
101,40184400,CI.GMA.,5.090,0.4,11.200,0.4,nan,0.3,b,20220212232629910
102,40184400,CI.LUC2.,7.300,0.4,12.100,0.4,nan,0.3,a,20220212232629910
103,40184400,AZ.SND.,12.400,0.4,15.100,0.4,nan,0.3,b,20220212232629910


In [48]:
evt_dir  = f"{ddir}/Download/events.txt"

events = pd.read_csv(evt_dir, header = None, delim_whitespace=True)


# Function to remove non-numerical symbols from a string
def remove_non_numerical(s):
    return re.sub('[^0-9]', '', s)

# Apply the function to the 'col1' column of the DataFrame
events[3] = events[3].apply(remove_non_numerical)
events

,0,1,2,3,4,5,6,7,8,9
0,39900807,eq,l,20220102001614080,32.2623,-115.3403,4.03,2.28,l,1.0
1,39901391,eq,l,20220102232745920,33.6480,-116.7418,15.41,2.36,l,1.0
2,39901519,eq,l,20220103023448200,35.2840,-119.3907,17.73,3.92,l,1.0
3,39902431,eq,l,20220104090504520,32.9600,-115.8628,7.95,2.00,l,1.0
4,39902479,eq,l,20220104124219810,34.1240,-117.5977,11.83,2.08,l,1.0
...,...,...,...,...,...,...,...,...,...,...
509,40292048,eq,l,20220627041323370,33.8412,-118.5292,10.77,2.27,l,1.0
510,40292088,eq,l,20220627065127410,32.6447,-115.7207,9.34,2.07,l,1.0
511,40292496,eq,l,20220627195131740,35.6527,-117.4580,1.33,2.10,l,1.0
512,40293376,eq,l,20220629041936690,33.1813,-115.6073,2.96,3.02,l,1.0


In [24]:
data_sac_b = "/home/youzhi/python-projects/sp-project/Data/BAD_SAC_for_readin"
data_sac_g = "/home/youzhi/python-projects/sp-project/Data/GOOD_SAC_for_readin"
switch = True 

In [ ]:
# testing if the TsPmP values need to add origin time (sachd["o"])
if switch == True:

    data_sac = data_sac_g #good signals
else: 
    data_sac = data_sac_b #bad signals


for year in sorted(os.listdir(data_sac)):
    #print(year)
    inpath = os.path.join(data_sac, year)
    if os.path.isdir(inpath):
        #print(inpath)
        os.chdir(inpath)
        for bp in sorted(glob.glob("*.ev")):
            #print(bp)

            # calculate the envelope
            tr = obspy.read(bp)
            print(tr[0].stats.sac["nevid"])

In [41]:
# this code plot will need to be run twice, once for bad and once for good
# remember to change the switch to False for the second run
if switch == True:

    data_sac = data_sac_g #good signals
else: 
    data_sac = data_sac_b #bad signals


for year in sorted(os.listdir(data_sac)):
    print(year)
    inpath = os.path.join(data_sac, year)
    if os.path.isdir(inpath):
        #print(inpath)
        os.chdir(inpath)
        for bp in sorted(glob.glob("*.bp.cut")):
            #print(bp)

            # calculate the envelope
            tr = obspy.read(bp)

            trenv = obspy.signal.filter.envelope(tr[0].data)
            
            if data_sac == data_sac_g:
                #good signals

                # add event id to ev
                evt_index = df_verified.loc[df_verified["evt_num"] == year].index[0]
                tr[0].stats.sac['nevid'] = str(df_verified["evt_id"][evt_index])

                #write sP arrival
                TsPmP = df_verified["TsPmP"][evt_index]
                Tp = df_verified["Tp"][evt_index]
                tr[0].stats.sac["t2"] = TsPmP - Tp #+ sachd["o"]
                
                tr[0].stats.sac["kt2"] = "1"
            else: 
                # bad signals
                tr[0].stats.sac["kt2"] = "0"
                evt_index = events.loc[events[3] == year].index[0]
                tr[0].stats.sac['nevid'] = str(events[0][evt_index])
                sachd = tr[0].stats.sac
                distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
                gcarc = distaz['distance']
                arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                              distance_in_degree=gcarc, phase_list=['sPvmP'])

                TsPmP = arrivals[0].time
                tr[0].stats.sac["t2"] = TsPmP


            # normalize the envelope
            trenv = trenv / np.max(trenv)
            tr[0].data = trenv
            

            # save ev file
            fname = year+'.' + tr[0].stats.network + '.' + tr[0].stats.station + tr[0].stats.location + '.BHZ'
            tr[0].write(fname +".ev", format='SAC')
            nbp = "{}.{}".format(fname,"ev")
            ntr = obspy.read(nbp)

            # cut the waveform within a given time window, -2 to 11 s
            # changed to 11 based on data I collected
            T = get_sac_reftime(ntr[0].stats.sac) + 0
            ntr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            ntr[0].write(nbp, format='SAC')

            # tr = obspy.read(bp)
            # # cut the waveform within a given time window, -2 to 11 s
            # T = get_sac_reftime(tr[0].stats.sac) + 0
            # tr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            # tr[0].write(bp, format='SAC')

20220112002458770
20220112021208390
20220112090603170
20220113032108850
20220113032453520
20220113082309790


/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39909423
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39909511
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39909663
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39910287
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39910327
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/p

20220115161902000
20220116091814410
20220116124930000
20220116145950590


/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39913455
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39913895
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39913943
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39913999
  warnings.warn(msg.format(hdr, value))


20220117000229990
20220117155620910
20220118073915120


...

/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39923623
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39923679
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39924543
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39924855
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 39926031
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/p

20220203075810850
20220203151832540
20220203170822680
20220204105219770
20220205074104630
20220205074909660
20220205183420810
20220205234904960
20220206232447510
20220207121028870
20220209014742470
20220209040551690


/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40175136
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 37387020
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40175560
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40176336
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40177192
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/p

20220209121834630
20220211121842170
20220211141550450
20220211142113660
20220211232548160
20220212232629910
20220214164135260


/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40180784
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40182952
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40183048
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40183056
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/pmpnet/lib/python3.9/site-packages/obspy/io/sac/arrayio.py:511: UserWarning: Non-integers may be truncated: nevid = 40183624
  warnings.warn(msg.format(hdr, value))
/home/youzhi/miniconda3/envs/p

# Move ev files into a new folder

In [28]:
target_dir_b = "/home/youzhi/python-projects/sp-project/Data/bad_readindata"
target_dir_g = "/home/youzhi/python-projects/sp-project/Data/good_readindata"
# good_readindata for good data
# bad_readindata for bad data

# move all the ev files into a folder
data_sac_b = "/home/youzhi/python-projects/sp-project/Data/BAD_SAC_for_readin"
data_sac_g = "/home/youzhi/python-projects/sp-project/Data/GOOD_SAC_for_readin"

In [45]:
if switch == True:

    data_sac = data_sac_g
    target_dir = target_dir_g
else: 
    data_sac = data_sac_b
    target_dir = target_dir_b

num_files = 0
for path in os.listdir(data_sac):
    #print(path)
    if path.endswith("ini") != True:
        
        for file in os.listdir(os.path.join(data_sac, path)):
            #print(file)
            if file.endswith(".ev"):

                #delete rundundant files from previous runs
                #os.remove(os.path.join(data_sac, path, file))
                filepath = os.path.join(data_sac, path, file)
                shutil.copy(filepath, os.path.join(target_dir, file))
                #print(filepath)
	    #some times if might count desktop.ini files
                num_files += 1
print(num_files)

105


# Combine envelope for neural network readin

In [9]:
datadir_b = "/home/youzhi/python-projects/sp-project/Data/bad_readindata"
datadir_g = "/home/youzhi/python-projects/sp-project/Data/good_readindata"
quality = 0

signal_evid = []
signal_evlo = []
signal_evla = []
signal_stlo = []
signal_stla = []
signal_fname = []
envelop_signal = []
PmP_label = []
PmP_time = []
PmP_ptime = []
PmP_dist = []
PmP_evdp = []
PmP_mag = []
j=0
for file in os.listdir(datadir_b):
    if file.endswith(".ev"):
        j=j+1
        st = read(os.path.join(f"{datadir_b}", file))
        envelop_signal.append(st[0].data)
        filenm = file.split('.',6)
        filenmall=f"{filenm[0]}"+'.'+f"{filenm[1]}"+'.'+f"{filenm[2]}"
        signal_evid.append(st[0].stats.sac.nevid)
        signal_evlo.append(st[0].stats.sac.evlo)
        signal_evla.append(st[0].stats.sac.evla)
        signal_stlo.append(st[0].stats.sac.stlo)
        signal_stla.append(st[0].stats.sac.stla)
        signal_fname.append(filenmall)
        if st[0].stats.sac.get("kt2") != None:
            if quality == 0:
                PmP_label.append(0) # 0 for bad, 1 for good
                #print("1")
            else:
                PmP_label.append(1)
                #print("0")
        else:
            PmP_label.append(0)
        PmP_time.append(st[0].stats.sac.t2)       ### differential traveltime between P and PmP waves
        PmP_ptime.append(st[0].stats.sac.o)       ### (negative) traveltime of first P wave (This is because waveform has been aligned to P-wave arrival)
        PmP_dist.append(st[0].stats.sac.dist)
        PmP_evdp.append(st[0].stats.sac.evdp)
        PmP_mag.append(st[0].stats.sac.mag)
        #print("%d  %s  %g  %g  %g  %g  %g %d"%(j,signal_fname[j-1],PmP_time[j-1],PmP_ptime[j-1],PmP_dist[j-1],PmP_evdp[j-1],PmP_mag[j-1],PmP_label[j-1]))

# save into a single file

In [43]:
def saveVariables(variables, filename): #where 'variables' is a list of variables
    with open(filename, 'wb+') as file:
        pickle.dump(variables, file)

In [10]:
#fast save and retrieve data, no need to read in data again(very slow)

def saveVariables(variables, filename): #where 'variables' is a list of variables
    with open(filename, 'wb+') as file:
        pickle.dump(variables, file)

#save the data
saveVariables([envelop_signal, PmP_time, PmP_label, PmP_dist, PmP_evdp, 
               PmP_mag, signal_stlo, signal_stla, signal_evlo, signal_evla, signal_evid, signal_fname], f"{datadir_b}/b_readindata")


In [ ]:
saveVariables([envelop_signal, PmP_time, PmP_label, PmP_dist, PmP_evdp, 
               PmP_mag, signal_stlo, signal_stla, signal_evlo, signal_evla, signal_evid, signal_fname], f"{datadir}/g_readindata")

# move the readin file into combined folder

In [11]:
target_dir = "/home/youzhi/python-projects/sp-project/Data/combined_readindata"

data_dir = "/home/youzhi/python-projects/sp-project/Data/bad_readindata"

filename = os.path.join(data_dir, "b_readindata")
shutil.copy(filename, os.path.join(target_dir, "b_readindata"))
#copy filename into target_dir with new name


'/home/youzhi/python-projects/sp-project/Data/combined_readindata/b_readindata'

# Combine bad and good data

In [79]:
data_dir =  "/home/youzhi/python-projects/sp-project/Data/combined_readindata"
spdir = "/home/youzhi/python-projects/sp-project/Data"

def retrieveVariables(filename):
    variables = []
    with open(str(filename), 'rb') as file:
        variables = pickle.load(file)
    return variables


# read in data

[envelop_signal, PmP_time, PmP_label, 
 PmP_dist, PmP_evdp, PmP_mag, PmP_stlo, 
 PmP_stla, PmP_evlo, PmP_evla, PmP_evid, 
 PmP_fname] = retrieveVariables(f"{data_dir}/g_readindata")

[envelop_signal2, PmP_time2, PmP_label2, 
 PmP_dist2, PmP_evdp2, PmP_mag2, PmP_stlo2, 
 PmP_stla2, PmP_evlo2, PmP_evla2, PmP_evid2, 
 PmP_fname2] = retrieveVariables(f"{data_dir}/b_readindata")

# randomly pick 300 indexes between 0 and 4000

list1 = [envelop_signal, PmP_time, PmP_label, 
 PmP_dist, PmP_evdp, PmP_mag, PmP_stlo, 
 PmP_stla, PmP_evlo, PmP_evla, PmP_evid, 
 PmP_fname] 

indexes = np.random.choice(4000, 300, replace=False)

list2 = [envelop_signal2, PmP_time2, PmP_label2, 
 PmP_dist2, PmP_evdp2, PmP_mag2, PmP_stlo2, 
 PmP_stla2, PmP_evlo2, PmP_evla2, PmP_evid2, 
 PmP_fname2]

list3 = []

for i in range(len(list2)):
    list2[i] = np.array(list2[i])[indexes]
    
    list3.append(np.concatenate((list2[i], list1[i])))

saveVariables(list3, f"{spdir}/combined_readindata/c_readindata")

# envelop_signal_c = np.concatenate((envelop_signal2, envelop_signal))
# PmP_time_c = np.concatenate((PmP_time2, PmP_time))
# PmP_label_c = np.concatenate((PmP_label2, PmP_label))
# PmP_dist_c= np.concatenate((PmP_dist2, PmP_dist))
# PmP_evdp_c= np.concatenate((PmP_evdp2, PmP_evdp))
# PmP_mag_c= np.concatenate((PmP_mag2, PmP_mag))
# PmP_stlo_c = np.concatenate((PmP_stlo2, PmP_stlo))
# PmP_stla_c= np.concatenate((PmP_stla2, PmP_stla))
# PmP_evlo_c= np.concatenate((PmP_evlo2, PmP_evlo))
# PmP_evla_c= np.concatenate((PmP_evla2, PmP_evla))
# PmP_evid_c= np.concatenate((PmP_evid2, PmP_evid))
# PmP_fname_c= np.concatenate((PmP_fname2, PmP_fname))


# saveVariables([envelop_signal_c, PmP_time_c, PmP_label_c, 
#  PmP_dist_c, PmP_evdp_c, PmP_mag_c, PmP_stlo_c, 
#  PmP_stla_c, PmP_evlo_c, PmP_evla_c, PmP_evid_c, 
#  PmP_fname_c], 
#               f"{spdir}/combined_readindata/c_readindata")

In [75]:
len(list3[0])

601